# 6. Advanced Modeling – Random Forest e Comparação de Modelos

Neste notebook treinamos um segundo modelo (Random Forest)
e comparamos seu desempenho com a Regressão Logística.

Objetivo: identificar o modelo com melhor capacidade de previsão de churn.


In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [0]:
df = spark.table("`crisp-dm`.gold_churn")

display(df)


In [0]:
df_model = df.select(
    "recency",
    "frequency",
    "monetary",
    "churn"
)


In [0]:
assembler = VectorAssembler(
    inputCols=["recency", "frequency", "monetary"],
    outputCol="features"
)

df_vector = assembler.transform(df_model)


In [0]:
train_df, test_df = df_vector.randomSplit([0.7, 0.3], seed=42)


In [0]:
lr = LogisticRegression(
    featuresCol="features",
    labelCol="churn"
)

lr_model = lr.fit(train_df)

lr_predictions = lr_model.transform(test_df)


In [0]:
rf = RandomForestClassifier(
    featuresCol="features",
    labelCol="churn",
    numTrees=100,
    maxDepth=5,
    seed=42
)

rf_model = rf.fit(train_df)

rf_predictions = rf_model.transform(test_df)


In [0]:
evaluator = BinaryClassificationEvaluator(
    labelCol="churn",
    metricName="areaUnderROC"
)

lr_auc = evaluator.evaluate(lr_predictions)
rf_auc = evaluator.evaluate(rf_predictions)

print("AUC Logistic Regression:", lr_auc)
print("AUC Random Forest:", rf_auc)


## Comparação de Modelos

- AUC Logistic Regression: 0.99
- AUC Random Forest: 0.99

O modelo com maior AUC apresenta melhor capacidade
de discriminação entre clientes churn e ativos.


In [0]:
import pandas as pd

feature_importance = rf_model.featureImportances.toArray()

features = ["recency", "frequency", "monetary"]

importance_df = pd.DataFrame({
    "Feature": features,
    "Importance": feature_importance
}).sort_values(by="Importance", ascending=False)

importance_df


## Importância das Variáveis

A Random Forest indica quais variáveis mais influenciam o churn.

Variáveis com maior importance possuem maior impacto
na decisão do modelo.

Isso auxilia na definição de estratégias de retenção.
